Tile RMS
====
Plot the RMS of the $8\times8\degree$ MDT current tiles, to visualise how the energy in the tiles varies - we want to exclude the regions near coastlines when picking our training data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib

# This is the "real" file the Laura made plots of in her paper doi:10.1017/eds.2023.41
filepath = pathlib.Path(
    "/home/mh19137/geog_rdsf/data/projects/SING/richard_stuff/Table2/currents/dtu18_eigen-6c4_do0280_rr0004_cs.dat"
)
assert filepath.exists()

In [ ]:
from current_denoising.generation import ioutils

data = ioutils.read_currents(filepath)

In [ ]:
import matplotlib.pyplot as plt

from current_denoising.utils import util
from current_denoising.plotting import maps

fig, axis = plt.subplots(1, 1, figsize=(12, 6))

# This isn't a great colourmap but it makes the plot match up with Figure 3 in Laura's paper
imshow_kw = {"origin": "upper", "vmin": 0, "vmax": 1.4, "cmap": "turbo"}
im = axis.imshow(data, **imshow_kw)

axis.set_title(filepath.stem)
fig.colorbar(im)

lat, long = util.lat_long_grid(data.shape)
extent = [long[0], long[-1], lat[0], lat[-1]]
im.set_extent(extent)

fig.tight_layout()

To illustrate how the power (RMS) of the tiles varies, we can make a plot...

In [ ]:
"""Plot the RMS of every possible patch in the image"""

import numpy as np
from tqdm.notebook import tqdm

tile_size = 32

# Get the location of the tiles
lat_indices = np.arange(*ioutils._included_indices(data.shape[0], tile_size, np.inf))
long_indices = np.arange(0, data.shape[1] - tile_size + 1)

# Init an empty array
tile_rms = np.ones_like(data) * np.nan

# Iterate over the tiles
pbar = tqdm(total=len(lat_indices) * len(long_indices))

for i in lat_indices:
    for j in long_indices:
        tile = util.tile(data, (i, j), tile_size)
        tile_rms[i + tile_size // 2, j + tile_size // 2] = ioutils.tile_rms(tile)
        pbar.update(1)

In [ ]:
fig, axes = plt.subplot_mosaic("""
                               AAAA
                               AAAA
                               BBB.
                               """, figsize=(12, 9))
im = axes["A"].imshow(tile_rms, **imshow_kw)
im.set_extent([long[0], long[-1], lat[0], lat[-1]])

_, bins, patches = axes["B"].hist(tile_rms.ravel(), bins=100)
percentile = 0.50
threshold = np.nanquantile(tile_rms.ravel(), percentile)
axes["B"].axvline(threshold, color="red")

bin_centres = 0.5 * (bins[1:] + bins[:-1])
for c, p in zip(bin_centres, patches):
    p.set_facecolor(im.cmap(im.norm(c)))
    p.set_edgecolor("none")

axes["A"].set_title(f"Tile RMS, {tile_size=}")

axes["B"].set_title(
    f"{percentile*100:.0f}"
    "$^{\mathrm{th}}$ percentile indicated"
    f" ({threshold:.2f}m/s)"
)
axes["B"].set_xlabel("Tile RMS")
axes["B"].set_ylabel("Count")

fig.colorbar(im, ax=axes["A"])

# Noise application

We estimate noise strength by creating a noise strength map - visualise this here

In [ ]:
from current_denoising.generation import applying_noise

filter_size = 10

filtered = applying_noise.noise_strength_map(data, filter_size=filter_size)

In [ ]:
fig, axis = plt.subplots(1, 1, figsize=(12, 6))

im = axis.imshow(filtered, **imshow_kw)

earth_radius_km = 6370
axis.set_title(
    f"{filter_size=} grid point\n"
    r"$\approx$"
    f"{earth_radius_km / 12 * filter_size:.0f}km at equator"
)
fig.colorbar(im)

lat, long = util.lat_long_grid(data.shape)
extent = [long[0], long[-1], lat[0], lat[-1]]
im.set_extent(extent)

fig.tight_layout()

In [ ]:
"""Plot a tile near Australia to reproduce Fig 9 in Laura's paper"""

import numpy as np


lat, long = 8.25, -76
size = 32

tile = util.get_tile(filtered, (lat, long), size)

fig, axis = plt.subplots()
axis.imshow(tile, **imshow_kw)
axis.set_axis_off()

fig.tight_layout()